In [1]:
import cv2
import numpy as np
import keras

In [2]:
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

In [3]:
model = keras.models.load_model('model_val67/model_fer2013_val67.h5')  ##Good with happy.

In [4]:
# # load json and create model
# # Good with happy,sad and default
# json_file = open('model_dwnd/emotion_model.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# model = keras.models.model_from_json(loaded_model_json)

# # load weights into new model
# model.load_weights("model_dwnd/emotion_model.h5")
# print("Loaded model from disk")

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 46, 46, 64)        18496     
                                                                 
 batch_normalization (BatchN  (None, 46, 46, 64)       256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 23, 23, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 23, 23, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 23, 23, 128)       2

In [ ]:
# Testing Face Detection
cap = cv2.VideoCapture(0)
#cap = cv2.VideoCapture("video.mp4")

while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame, (1280, 720))
    if not ret:
        break
    face_detector = cv2.CascadeClassifier('haar_cascade/haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)
    
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cv2.imshow('Grayface',roi_gray_frame)
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)
        cropped_img = cropped_img.astype("float")/255.0
        emotion_prediction = model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        
        
    cv2.imshow('Emotion Detection', frame)
    if cv2.waitKey(1)==ord('q'):
            break
            

cap.release()
cv2.destroyAllWindows()